In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
import numpy as np, random
import pandas as pd 
from src.config import Config
from src.insights import get_training_dfs
from src.utils import display_all
import wandb
from pandas.io.json._normalize import nested_to_record    

seed = 100
np.random.seed(seed)

2023-04-14 09:28:53.690918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 09:29:16.134196: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:/usr/local/cuda-11.6/extras/CUPTI/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-11.6/lib64:/usr/local/cuda-11.6/extras/CUPTI/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64::/home/tproth/Programs/miniconda/lib/:/home/tproth/Data/miniconda3/lib/:/home/tproth/Programs/miniconda/lib/:/home/tproth/Data/minico

Parameters: (dataset_name, run_name)

Csv path: f"{self._cfg.path_run}{split}.csv"
Initial values are stored at 


Get names of all runs that differ only in seed 
    * kind of like: take run_results table, filter to only date you are interested in, group by (everything else than seed) and then concat the run_name to get a list. Then save the list. Now you can enumerate on it. 

create csv bootstrap_results_{datetime}.csv
for run_list in run_lists
    Init empty list of tables
    For run in run_list: 
        load + process test table 
        concat to list 
    concat all list of tables to make one big table 
    run bootstrap test on big table 
    save p-value with run names and common config to bootstrap_results 


In [4]:
api = wandb.Api()
runs = api.runs("uts_nlp/travis_attack", filters={"tags": {"$in": ["final"]}})
summary_params = ['baseline_test', 'any_adv_example_proportion-test',  'is_adv_example-mean-test', 'n_pp-test']
config_params = ['decode_method_eval', 'dataset_name', 'seed', 'gen_params_train']


summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    d = run.summary._json_dict
    d1 = nested_to_record({ k: d[k] for k in summary_params })
    l = list(d1.keys())
    for k in l: 
        if 'baseline_test' in k: 
            if k not in ['baseline_test.any_adv_example_proportion', 'baseline_test.is_adv_example-mean']: d1.pop(k)
    summary_list.append(d1)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    d1 = nested_to_record({k: v for k,v in run.config.items() if k in config_params})
    for k in ['gen_params_train.top_p','gen_params_train.do_sample']: d1.pop(k)
    #d2 = {k: v for k,v in run.config.items() if k in config_params}
    config_list.append(d1)

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
   "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

res_df = pd.concat(objs=[runs_df['name'], pd.DataFrame.from_records(runs_df["config"]), pd.DataFrame.from_records(runs_df["summary"])], axis=1)
res_df['avg_num_successes'] = res_df['is_adv_example-mean-test'] * res_df['n_pp-test']
res_df['decode_method_eval']  = pd.Categorical(res_df['decode_method_eval'], 
                      categories=["sample","beam_search","diverse_beam_search_low_diversity","diverse_beam_search_high_diversity"], ordered=False)
res_df['dataset_name']  = pd.Categorical(res_df['dataset_name'], 
                      categories=["rotten_tomatoes","financial"], ordered=False)

res_df = res_df.sort_values(['dataset_name', 'decode_method_eval', 'gen_params_train.temperature'])
#res_df
def replace_names(df): 
    df['dataset_name'].replace({"rotten_tomatoes": "RT", "financial":"FP" }, inplace=True)
    df['decode_method_eval'].replace({"sample": "Sample", "beam_search":"Beam search",
                                         "diverse_beam_search_low_diversity": "Diverse beam search (6 beam groups)",
                                         "diverse_beam_search_high_diversity": "Diverse beam search (48 beam groups)"}, inplace=True)
    df['decode_method_eval']  = pd.Categorical(df['decode_method_eval'], 
                        categories=["Sample","Beam search","Diverse beam search (6 beam groups)","Diverse beam search (48 beam groups)"], ordered=False)
    df['dataset_name']  = pd.Categorical(df['dataset_name'], 
                        categories=["RT","FP"], ordered=False)
    return df 
res_df = replace_names(res_df)

In [11]:
res_df.head(2)

,name,seed,dataset_name,decode_method_eval,gen_params_train.temperature,any_adv_example_proportion-test,is_adv_example-mean-test,n_pp-test,baseline_test.is_adv_example-mean,baseline_test.any_adv_example_proportion,avg_num_successes
45,proud-sweep-3,1002,RT,Sample,0.85,0.339833,0.056066,30.395543,0.024213,0.306407,1.704161
46,ruby-sweep-2,1001,RT,Sample,0.85,0.384401,0.093902,28.231198,0.023408,0.284123,2.650964


In [18]:
## get max values per condition (including baseline)
res_df.groupby(['dataset_name', 'decode_method_eval', 'gen_params_train.temperature']).agg({
     'baseline_test.any_adv_example_proportion': 'max', 'any_adv_example_proportion-test': 'max'}).round(3)


baseline_test.any_adv_example_proportion  \
dataset_name decode_method_eval                   gen_params_train.temperature                                             
RT           Sample                               0.85                                                             0.306   
                                                  1.15                                                             0.306   
             Beam search                          0.85                                                             0.145   
                                                  1.15                                                             0.145   
             Diverse beam search (6 beam groups)  0.85                                                             0.206   
                                                  1.15                                                             0.206   
             Diverse beam search (48 beam groups) 0.85                                                             0.245   
                                                  1.15                                                             0.245   
FP           Sample                               0.85                                                             0.208   
                                                  1.15                                                             0.208   
             Beam search                          0.85                                                             0.113   
                                                  1.15                                                             0.113   
             Diverse beam search (6 beam groups)  0.85                                                             0.132   
                                                  1.15                                                             0.132   
             Diverse beam search (48 beam groups) 0.85                                                             0.214   
                                                  1.15                                                             0.214   

                                                                                any_adv_example_proportion-test  
dataset_name decode_method_eval                   gen_params_train.temperature                                   
RT           Sample                               0.85                                                    0.418  
                                                  1.15                                                    0.468  
             Beam search                          0.85                                                    0.855  
                                                  1.15                                                    0.855  
             Diverse beam search (6 beam groups)  0.85                                                    0.432  
                                                  1.15                                                    0.577  
             Diverse beam search (48 beam groups) 0.85                                                    0.660  
                                                  1.15                                                    0.415  
FP           Sample                               0.85                                                    0.723  
                                                  1.15                                                    0.836  
             Beam search                          0.85                                                    0.843  
                                                  1.15                                                    0.881  
             Diverse beam search (6 beam groups)  0.85                                                    0.824  
                                                  1.15                                                    0.824  
             Diverse be

dataset_name  gen_params_train.temperature  decode_method_eval                  
RT            0.85                          Sample                                  47
                                            Beam search                              0
                                            Diverse beam search (6 beam groups)     36
                                            Diverse beam search (48 beam groups)    29
              1.15                          Sample                                  43
                                            Beam search                             39
                                            Diverse beam search (6 beam groups)     33
                                            Diverse beam search (48 beam groups)    26
FP            0.85                          Sample                                  25
                                            Beam search                             17
                                            Diver

In [40]:
## Run lists 
cfg = Config()
#df_runs = pd.read_csv(f"{cfg.path_results}run_results.csv")
#df_runs1 = df_runs[255:]  # filter which rows you want by eyeballing (usually these are part of a sweep)
#df_config = df_runs1[['run_name','seed', 'dataset_name', 'gen_params_train.temperature', 'decode_method_eval']].drop_duplicates()


# CONFIG1: Get all runs per condition
df_config  = res_df.groupby(['dataset_name', 'gen_params_train.temperature', 'decode_method_eval'])['name']\
                     .apply(list).to_frame('run_names').reset_index()

# CONFIG2: only select the max run for each condition
# # Groupby condition and extract the name of the run with the max value
df_config1 = res_df.groupby(['dataset_name', 'gen_params_train.temperature', 'decode_method_eval']).apply(
    lambda x: x.loc[x['any_adv_example_proportion-test'].idxmax(), 'name']).to_frame('run_names').reset_index()

In [45]:
def load_and_process_run_results(cfg, run_name, split="test"):
    # Load results
    df = get_training_dfs(f"{cfg.path_checkpoints}{run_name}/", postprocessed=False)[split]
    if split != "test": 
        min_epoch,max_epoch = min(df['epoch']),max(df['epoch'])
        df = df.query("epoch==@min_epoch or epoch==@max_epoch")
    # Agg to one row per orig example
    df_any_adv_example = df.groupby(['idx', 'epoch'])['is_adv_example'].agg('sum').apply(lambda x: (x > 0)*1).reset_index()
    # Setup baseline and trained conditions and pivot to get results
    df_any_adv_example['condition'] = df_any_adv_example['epoch'].apply(lambda x: "baseline" if x==0 else "trained") 
    df_any_adv_example.drop(columns='epoch', inplace=True)
    df_wide = df_any_adv_example.pivot(index='idx',  columns=['condition'])
    df_wide.columns = ["_".join(a) for a in df_wide.columns.to_flat_index()]
    df_wide.index = [f"{run_name}_{c}" for c in df_wide.index]
    return df_wide

def bootstrap_from_pd(df, b, A_col, B_col): 
    diff_overall =  df[A_col].sum() - df[B_col].sum()
    diff_l = []
    for i in range(b): 
        df_sample = df.sample(n=df.shape[0], replace=True)
        diff_l.append(df_sample[A_col].sum() - df_sample[B_col].sum())
   # print(diff_overall)
   # print(diff_l)
    condition_met = sum([ (diff_sample > (2 * diff_overall))*1  for diff_sample in diff_l])
    p_val = condition_met/b      
    return p_val

# Bootstrap 
b = 10
split="test"
for index, row in df_config1.iterrows(): 
    run_list = row['run_names']
    if type(run_list) is not list: run_list = [run_list]
    print([o[1] for o in row.items()])
    df_l = []
    for run_name in run_list:
        df_l.append(load_and_process_run_results(cfg, run_name, split=split))
    df_examples = pd.concat(df_l)
    p_val = bootstrap_from_pd(df_examples, b=b, A_col="is_adv_example_trained", B_col = "is_adv_example_baseline")
    print(p_val)
    print("\n########################\n")

['RT', 0.85, 'Sample', 'earthy-sweep-1']
0.0

########################

['RT', 0.85, 'Beam search', 'zany-surf-809']
0.0

########################

['RT', 0.85, 'Diverse beam search (6 beam groups)', 'silvery-sweep-14']
0.0

########################

['RT', 0.85, 'Diverse beam search (48 beam groups)', 'ancient-sweep-21']
0.0

########################

['RT', 1.15, 'Sample', 'deep-sweep-5']
0.0

########################

['RT', 1.15, 'Beam search', 'flowing-sweep-10']
0.0

########################

['RT', 1.15, 'Diverse beam search (6 beam groups)', 'dashing-sweep-17']
0.0

########################

['RT', 1.15, 'Diverse beam search (48 beam groups)', 'grateful-sweep-24']
0.0

########################

['FP', 0.85, 'Sample', 'revived-sweep-25']
0.0

########################

['FP', 0.85, 'Beam search', 'polished-sweep-31']
0.0

########################

['FP', 0.85, 'Diverse beam search (6 beam groups)', 'comfy-sweep-36']
0.0

########################

['FP', 0.85, 'Diverse beam search

In [ ]:

## THE ORIGINAL BOOTSTRAP FUNCTION - uncomment to see example of how to use it

# def bootstrap(set_X, b): #returns p-value(x)
#     d_X = np.sum(list(zip(*set_X))[0]) - np.sum(list(zip(*set_X))[1]) # how much better does algorithm A do than B on x
#     d_X_1tob = [] 
#     for i in range(0, b):
#         A1_b, B1_b = (0, 0)
#         # Draw a bootstrap sample x(i) of size n
#         for j in range(len(set_X)):
#             #Select a member of x at random and add it to x(i)
#             set_Xb = random.choice(set_X) 
#             A1_b += set_Xb[0]
#             B1_b += set_Xb[1]
#         d_X_1tob.append(A1_b - B1_b)  #delta: how much better does algorithm A do than B on x(i)

#     #Count the samples on which algorithm A accidentally did better than B
#     s = 0  
#     for dx in d_X_1tob:
#         if dx > (2 * d_X):
#             s += 1    

#     #onesided empirical p-value 
#     p_val = s/b      
#     return p_val
# bstrap_input = [(1, 1),
#  (1, 1),
#  (0, 0),
#  (0, 0)]
# print(bootstrap(bstrap_input, b=100))